In [1]:
from os.path import abspath, join

from run_attacks_and_defenses import read_submission, load_defense_output
from run_attacks_and_defenses import DatasetMetadata


use_gpu = False
epsilon = 16

images_path = abspath('dataset/images')
meta_path = abspath('dataset/dev_dataset.csv')
attack_path = abspath('sample_attacks/fgsm')
attack_output_path = abspath('attack_output')

defense_path = abspath('sample_defenses/adv_inception_v3')
defense_output_path = abspath('defense_output')

In [2]:
attack = read_submission(attack_path, use_gpu)
attack.run(images_path, attack_output_path, epsilon)

Running attack  fgsm
docker run -v /Users/dmitryka/nips/adversarial/dataset/images:/input_images -v /Users/dmitryka/nips/adversarial/attack_output:/output_images -v /Users/dmitryka/nips/adversarial/sample_attacks/fgsm:/code -w /code goodfellow/competition:cpu ./run_attack.sh /input_images /output_images 16


In [3]:
defense = read_submission(defense_path, use_gpu)
defense.run(attack_output_path, defense_output_path)

Running defense  adv_inception_v3
docker run -v /Users/dmitryka/nips/adversarial/attack_output:/input_images -v /Users/dmitryka/nips/adversarial/defense_output:/output_data -v /Users/dmitryka/nips/adversarial/sample_defenses/adv_inception_v3:/code -w /code gcr.io/tensorflow/tensorflow:1.1.0 ./run_defense.sh /input_images /output_data/result.csv


In [4]:
dataset_meta = DatasetMetadata(meta_path)
defense_result = load_defense_output(join(defense_output_path, 'result.csv'))

accuracy = 0
for image_id, predicted_label in defense_result.items():
    true_label = dataset_meta.get_true_label(image_id)
    if true_label == predicted_label:
        accuracy += 1
accuracy

812